In [1]:
using BenchmarkTools

In [2]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [3]:
function Convolution_2d_pad(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_pad (generic function with 1 method)

In [4]:
function Convolution_2d(input, kernel; bias=0.)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [5]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d_pad(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [6]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d_pad(tmp_weights, rotate(tmp_gradient); padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [7]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [8]:
input = rand(28, 28, 1);
weights = rand(3, 3, 1, 6);
bias = rand(Float32, 6);
gradient = rand(26, 26, 6);

In [9]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 722 samples with 1 evaluation.
 Range (min … max):  4.719 ms … 12.454 ms  ┊ GC (min … max):  0.00% … 51.26%
 Time  (median):     7.562 ms              ┊ GC (median):    34.68%
 Time  (mean ± σ):   6.920 ms ±  1.489 ms  ┊ GC (mean ± σ):  24.76% ± 17.77%

     ▅█▃                        ▁▃▂▂▄                         
  ▃▃▆████▆▅▄▄▃▂▃▃▂▂▂▁▁▁▁▁▂▁▂▁▁▂▇█████▇▇▅▅▅▅▅▃▄▃▃▂▃▂▃▂▃▃▃▁▃▂▂ ▃
  4.72 ms        Histogram: frequency by time          10 ms <

 Memory estimate: 25.79 MiB, allocs estimate: 4842.

In [ ]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

In [ ]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

In [ ]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)